In [ ]:
import os, re, pandas as pd, numpy as np, gensim, matplotlib.pyplot as plt, warnings, spacy
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
import base as bs

warnings.filterwarnings("ignore")

## Este codigo se encarga de realizar el NLP




data_aux = []
# iteramos sobre cada fila del dataframe



for i in range(len(data)):
    aux = []
    for item in list(bs.useLDA(texts = bs.pre_proc_text(df_ = data, tipo = 'Contenido', cliente_ = "Ecopetrol").loc[[i,]], n = 3)[0][1]):
        aux.append(item[0])
    data_aux.append(aux)
data['Topics'] = data_aux

In [ ]:
# Cargamos los datos del WS
data = bs.read_data("../data/raw/",['Fecha Extraccion', 'Titulo', 'Fecha Publicacion',
                               'Resumen', 'URL','Imagen', 'Empresa', 'Contenido',
                               'Tags', 'Tema', 'Autor','RelNewsUrls', 'Fuente'])

In [ ]:
# Hacemos el pre-procesamiento de los datos
data["DataProcess"] = bs.pre_proc_text(df_ = data, tipo = 'Contenido')


In [ ]:
# Obtenemos los topics
data['Topics'] = data[].apply(lambda x: bs.useLDA([x["DataProcess"]],n = 3) , axis=1)


In [ ]:
data

In [ ]:
# Guardamos los datos en curated
data.to_csv("../data/curated/curated_data.csv",index=False)